In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.src.legacy.preprocessing.image import  ImageDataGenerator
from keras.src.legacy.preprocessing.image import *
from keras.src.callbacks import EarlyStopping
from keras.src.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, auc
import seaborn as sns
import cv2
import pandas as pd
import time
from google.colab import files
import zipfile

# Configurações
IMG_SIZE = 225
BATCH_SIZE = 32
EPOCHS = 50


In [5]:
# Instale a biblioteca Kaggle
!pip install kaggle

# Crie o diretório e configure a chave
from google.colab import files
import os

# Upload do kaggle.json
uploaded = files.upload()  # Selecione o arquivo kaggle.json

# Mova o arquivo para o diretório correto e ajuste as permissões
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename(list(uploaded.keys())[0], '/root/.kaggle/kaggle.json')
!chmod 600 /root/.kaggle/kaggle.json

# Baixe o dataset (substitua pelo link do seu dataset)
!kaggle datasets download -d bmadushanirodrigo/fracture-multi-region-x-ray-data

# Descompacte o arquivo (se necessário)
!unzip fracture-multi-region-x-ray-data.zip

A saída de streaming foi truncada nas últimas 5000 linhas.
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3-rotated1 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3-rotated1.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2-rotated3.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated2.jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured/14-rotated2-rotated3 (1).jpg  
  inflating: Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classificatio

In [6]:
def preprocess_image(image):
    """
    Função para pré-processar imagens de prints de tela
    """


    # Garantir que a imagem está no formato correto (uint8) para equalizeHist
    image = image.astype(np.uint8)

    # Aplicar equalização de histograma para melhorar contraste
    image = cv2.equalizeHist(image)

    # Aplicar filtro gaussiano para reduzir ruído
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Normalizar para [0, 1]
    image = image.astype(np.float32) / 255.0

    # Redimensionar para o tamanho esperado
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    # Adicionar canal de cor para compatibilidade com a CNN
    image = np.expand_dims(image, axis=-1)

    return image

In [7]:
def create_model():
    """
    Cria o modelo CNN
    """
    model = models.Sequential([
        # Primeira camada convolucional
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        layers.MaxPooling2D((2, 2)),

        # Segunda camada convolucional
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Terceira camada convolucional
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        # Camadas densas
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [8]:
def plot_training_history(history):
    """
    Plota os gráficos de acurácia e perda durante o treinamento
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Gráfico de acurácia
    ax1.plot(history.history['accuracy'], label='Treino')
    ax1.plot(history.history['val_accuracy'], label='Validação')
    ax1.set_title('Acurácia durante o treinamento')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Acurácia')
    ax1.legend()

    # Gráfico de perda
    ax2.plot(history.history['loss'], label='Treino')
    ax2.plot(history.history['val_loss'], label='Validação')
    ax2.set_title('Perda durante o treinamento')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Perda')
    ax2.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

In [9]:
def plot_classification_metrics(y_true, y_pred, class_names, output_path='classification_metrics.png'):
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)
    macro = precision_recall_fscore_support(y_true, y_pred, average='macro')
    weighted = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    data = {
        'Categoria': [f'{class_names[0]} (Classe 0)', f'{class_names[1]} (Classe 1)', 'Média Macro', 'Média Ponderada'],
        'Precisão': [precision[0], precision[1], macro[0], weighted[0]],
        'Revocação': [recall[0], recall[1], macro[1], weighted[1]],
        'F1-Score': [f1[0], f1[1], macro[2], weighted[2]],
    }
    df = pd.DataFrame(data)
    df_melted = df.melt(id_vars='Categoria', var_name='Métricas', value_name='Pontuação')

    plt.figure(figsize=(10, 5))
    ax = sns.barplot(data=df_melted, x='Categoria', y='Pontuação', hue='Métricas')
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=10, color='black', xytext=(0, 3), textcoords='offset points')
    plt.ylim(0, 1.05)
    plt.title('Desempenho do Modelo por Classe e Métrica')
    plt.ylabel('Pontuação')
    plt.xlabel('Categorias')
    plt.legend(title='Métricas')
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()


In [10]:
def plot_roc_curve(y_true, y_score, output_path='roc_curve.png'):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color='orange', label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

In [ ]:
from PIL import Image
import os

# Configure o diretório_base para o seu conjunto de dados
diretorio_base = '/content/drive/MyDrive/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification'  # Substitua pelo caminho correto
subdiretorios = ['train', 'test', 'val'] # Adapte para os seus subdiretórios
formatos_imagem_validos = ['.jpg', '.jpeg', '.png', '.bmp'] # Adicione outros formatos se necessário

imagens_corrompidas = []

for sub in subdiretorios:
    caminho_sub = os.path.join(diretorio_base, sub)
    if not os.path.isdir(caminho_sub):
        print(f"Aviso: Subdiretório {caminho_sub} não encontrado.")
        continue
    for classe_img in os.listdir(caminho_sub):
        caminho_classe = os.path.join(caminho_sub, classe_img)
        if not os.path.isdir(caminho_classe):
            continue
        for nome_arquivo in os.listdir(caminho_classe):
            if any(nome_arquivo.lower().endswith(ext) for ext in formatos_imagem_validos):
                caminho_arquivo = os.path.join(caminho_classe, nome_arquivo)
                try:
                    img = Image.open(caminho_arquivo)
                    img.verify()  # Verifica a integridade do arquivo
                    # Tenta carregar os dados da imagem para forçar a leitura completa
                    # Isso é importante porque img.verify() pode não detectar todos os problemas

                except (IOError, SyntaxError, OSError) as e:
                    print(f'Imagem corrompida ou truncada encontrada: {caminho_arquivo} - {e}')
                    imagens_corrompidas.append(caminho_arquivo)

if imagens_corrompidas:
    print("\nLista de imagens corrompidas:")
    for img_path in imagens_corrompidas:
        print(img_path)
    # Você pode optar por remover as imagens corrompidas aqui:
    # for img_path in imagens_corrompidas:
    #     os.remove(img_path)
    # print("Imagens corrompidas foram removidas.")
else:
    print("Nenhuma imagem corrompida encontrada.")

Nenhuma imagem corrompida encontrada.


In [11]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [12]:
def main():
    # Iniciar o cronômetro
    tempo_inicio = time.time()

    # Carregar e pré-processar os dados
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_image,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_image
    )

    train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='grayscale'
    )

    validation_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='grayscale'
    )

    test_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        color_mode='grayscale',
        shuffle=False
    )

    # Criar e treinar o modelo
    model = create_model()

    callbacks = [
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
    ]

    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=callbacks
    )

    # Plotar histórico de treinamento
    plot_training_history(history)

    # Avaliar o modelo
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nAcurácia no conjunto de teste: {test_acc:.4f}')

    # Fazer previsões
    predictions = model.predict(test_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = test_generator.classes

    # Calcular métricas
    print('\nRelatório de Classificação:')
    print(classification_report(y_true, y_pred))

    # Plotar matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Matriz de Confusão')
    plt.ylabel('Valor Real')
    plt.xlabel('Valor Previsto')
    plt.savefig('confusion_matrix.png')
    plt.close()

    # Plotar gráfico de métricas por classe
    class_names = ['Normal', 'Pneumonia']
    plot_classification_metrics(y_true, y_pred, class_names, output_path='classification_metrics.png')

    # Plotar curva ROC
    plot_roc_curve(y_true, predictions, output_path='roc_curve.png')

    # Calcular e exibir o tempo total de treinamento
    tempo_fim = time.time()
    tempo_total = tempo_fim - tempo_inicio
    horas = int(tempo_total // 3600)
    minutos = int((tempo_total % 3600) // 60)
    segundos = int(tempo_total % 60)
    print(f'\nTempo total de treinamento: {horas:02d}:{minutos:02d}:{segundos:02d}')

if __name__ == '__main__':
    main()

Found 9246 images belonging to 2 classes.
Found 829 images belonging to 2 classes.
Found 506 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.5413 - loss: 0.6874

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


289/289 ━━━━━━━━━━━━━━━━━━━━ 141s 357ms/step - accuracy: 0.5415 - loss: 0.6873 - val_accuracy: 0.7672 - val_loss: 0.5590
Epoch 2/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 94s 324ms/step - accuracy: 0.6479 - loss: 0.6294 - val_accuracy: 0.7491 - val_loss: 0.5351
Epoch 3/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 139s 313ms/step - accuracy: 0.6787 - loss: 0.5933 - val_accuracy: 0.7624 - val_loss: 0.5180
Epoch 4/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 95s 328ms/step - accuracy: 0.6980 - loss: 0.5698 - val_accuracy: 0.7937 - val_loss: 0.4663
Epoch 5/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 90s 312ms/step - accuracy: 0.7183 - loss: 0.5544 - val_accuracy: 0.7612 - val_loss: 0.5053
Epoch 6/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 88s 306ms/step - accuracy: 0.7129 - loss: 0.5495 - val_accuracy: 0.7793 - val_loss: 0.4738
Epoch 7/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 139s 296ms/step - accuracy: 0.7435 - loss: 0.5232 - val_accuracy: 0.7865 - val_loss: 0.4567
Epoch 8/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 86s 299ms/step - accuracy: 0.7427 - loss: 0.5109 - 